In [1]:
import pandas as pd

covid_data = pd.read_csv('D:\IDE\data\covid_data.csv')
vaccinations_data = pd.read_csv('D:\IDE\data\country_vaccinations.csv')

vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]

covid_data = covid_data.groupby(
    ['date', 'country'],
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()


covid_data['date'] = pd.to_datetime(covid_data['date'])
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']
covid_data = covid_data.sort_values(by=['country', 'date'])

covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

covid_df = covid_data.merge(vaccinations_data, on=['date', 'country'], how='left')
covid_df['death_rate'] = covid_df['deaths'] / covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered'] / covid_df['confirmed'] * 100 

In [38]:
c_dc = covid_df.copy()
# covid_data.index = ['Порядковый_номер', ]

In [17]:
# c_dc.set_index(
#     'Дата', 'Страна','Подтвержденная_тест', 'Летальный_исход', 
#     'Выздоровил', 'Болеет', 'Количество_подтвержденных_в_день', 
#     'Ежедневная смертность', 'Выздоровело_в_день', 'Общее количество прививок',
#     'Людей вакционировалось', 'Вакционированных в процентном соотношении',
#     'Людей полностью вакционированных', 'Полностью вакционированные в процентах',
#     'Вакцин сделано в день', 'Вакцина', 'Смертность', 'Скорость восстановления')
c_dc.describe()
c_dc.head()

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines,death_rate,recover_rate
0,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [24]:
c_dc['country'].value_counts()

Taiwan                 494
Japan                  494
Thailand               494
South Korea            494
United States          494
                      ... 
Channel Islands          1
Cape Verde               1
North Ireland            1
East Timor               1
Republic of Ireland      1
Name: country, Length: 221, dtype: int64

In [35]:
c_dc['daily_confirmed'].isna().sum(0)

221

In [58]:
# for col in c_dc.columns:
#     pct_missing = c_dc[col].isna().mean()
#     print(f'{col} - {pct_missing :.1%}')

c_dc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86785 entries, 0 to 86784
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 86785 non-null  datetime64[ns]
 1   country                              86785 non-null  object        
 2   confirmed                            86785 non-null  float64       
 3   deaths                               86785 non-null  float64       
 4   recovered                            86785 non-null  float64       
 5   active                               86785 non-null  float64       
 6   daily_confirmed                      86564 non-null  float64       
 7   daily_deaths                         86564 non-null  float64       
 8   daily_recovered                      86564 non-null  float64       
 9   total_vaccinations                   10629 non-null  float64       
 10  people_vac

In [59]:
c_dc.groupby(['country','active']).agg(['median','max'])

C:\Users\Huawei\AppData\Local\Temp\ipykernel_9088\2550532183.py:1: FutureWarning: ['vaccines'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  c_dc.groupby(['country','active']).agg(['median','max'])


date             \
                                                    median        max   
country                        active                                   
Afghanistan                    1.0     2020-02-27 12:00:00 2020-03-02   
                               2.0     2020-03-03 00:00:00 2020-03-03   
                               4.0     2020-03-05 12:00:00 2020-03-07   
                               5.0     2020-03-08 00:00:00 2020-03-08   
                               7.0     2020-03-09 00:00:00 2020-03-09   
...                                                    ...        ...   
Zimbabwe                       10018.0 2021-01-14 00:00:00 2021-01-14   
                               10029.0 2021-01-15 00:00:00 2021-01-15   
                               10326.0 2021-01-16 00:00:00 2021-01-16   
occupied Palestinian territory 0.0     2020-03-14 12:00:00 2020-03-17   
                               25.0    2020-03-10 00:00:00 2020-03-10   

                                       confirmed          deaths         \
                                          median      max median    max   
country                        active                                     
Afghanistan                    1.0           1.0      1.0    0.0    0.0   
                               2.0           2.0      2.0    0.0    0.0   
                               4.0           4.0      4.0    0.0    0.0   
                               5.0           5.0      5.0    0.0    0.0   
                               7.0           7.0      7.0    0.0    0.0   
...                                          ...      ...    ...    ...   
Zimbabwe                       10018.0   25368.0  25368.0  636.0  636.0   
                               10029.0   26109.0  26109.0  666.0  666.0   
                               10326.0   26881.0  26881.0  683.0  683.0   
occupied Palestinian territory 0.0           0.0      0.0    0.0    0.0   
                               25.0         25.0     25.0    0.0    0.0   

                                       recovered          daily_confirmed  \
                                          median      max          median   
country                        active                                       
Afghanistan                    1.0           0.0      0.0             0.0   
                               2.0           0.0      0.0             1.0   
                               4.0           0.0      0.0             0.0   
                               5.0           0.0      0.0             1.0   
                               7.0           0.0      0.0             2.0   
...                                          ...      ...             ...   
Zimbabwe                       10018.0   14714.0  14714.0          1112.0   
                               10029.0   15414.0  15414.0           741.0   
                               10326.0   15872.0  15872.0           772.0   
occupied Palestinian territory 0.0           0.0      0.0             0.0   
                               25.0          0.0      0.0             NaN   

                                                ... people_fully_vaccinated  \
                                           max  ...                  median   
country                        active           ...                           
Afghanistan                    1.0         0.0  ...                     NaN   
                               2.0         1.0  ...                     NaN   
                               4.0         2.0  ...                     NaN   
                               5.0         1.0  ...                     NaN   
                               7.0         2.0  ...                     NaN   
...                                        ...  ...                     ...   
Zimbabwe                       10018.0  1112.0  ...                     NaN   
                               10029.0   741.0  ...                     NaN   
                               1